# <center>Implementing LDA in Python</center>

## Importing the Required Libraries

In [1]:
#https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#1introduction
import numpy as np
import json
import glob

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy
from nltk.corpus import stopwords

#vis
import pyLDAvis
# import pyLDAvis.gensim
import pyLDAvis.gensim_models as gensimvis

import string
import re, numpy as np, pandas as pd

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

/usr/lib/python3/dist-packages/past/types/oldstr.py:23: DeprecationWarning: invalid escape sequence '\d'
  """
/usr/lib/python3/dist-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload


## Preparing the Data

In [2]:
bg_stopwords = open("data/stopwords/bg.txt", "r").read().split("\n")
en_stopwords = open("data/stopwords/en.txt", "r").read().split("\n")
stopwords = bg_stopwords + en_stopwords
print(stopwords)

['а', 'автентичен', 'аз', 'ако', 'ала', 'бе', 'без', 'беше', 'би', 'бивш', 'бивша', 'бившо', 'бил', 'била', 'били', 'било', 'благодаря', 'близо', 'бъдат', 'бъде', 'бяха', 'в', 'вас', 'ваш', 'ваша', 'вероятно', 'вече', 'взема', 'ви', 'вие', 'винаги', 'внимава', 'време', 'все', 'всеки', 'всички', 'всичко', 'всяка', 'във', 'въпреки', 'върху', 'г', 'ги', 'главен', 'главна', 'главно', 'глас', 'го', 'година', 'години', 'годишен', 'д', 'да', 'дали', 'два', 'двама', 'двамата', 'две', 'двете', 'ден', 'днес', 'дни', 'до', 'добра', 'добре', 'добро', 'добър', 'докато', 'докога', 'дори', 'досега', 'доста', 'друг', 'друга', 'други', 'е', 'евтин', 'едва', 'един', 'една', 'еднаква', 'еднакви', 'еднакъв', 'едно', 'екип', 'ето', 'живот', 'за', 'забавям', 'зад', 'заедно', 'заради', 'засега', 'заспал', 'затова', 'защо', 'защото', 'и', 'из', 'или', 'им', 'има', 'имат', 'иска', 'й', 'каза', 'как', 'каква', 'какво', 'както', 'какъв', 'като', 'кога', 'когато', 'което', 'които', 'кой', 'който', 'колко', 'която

In [3]:
def load_data(file):
    with open (file, "r", encoding="utf-8") as f:
        data = json.load(f) 
    return (data)

def write_data(file, data):
    with open (file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)
        
def remove_stops(text, stops):
    text = re.sub(r"AC\/\d{1,4}\/\d{1,4}", "", text).lower()
    words = text.split()
    final = []
    for word in words:
        if word not in stops:
            final.append(word)
    final = " ".join(final)
    final = final.translate(str.maketrans("", "", string.punctuation))
    final = "".join([i for i in final if not i.isdigit()])
    while "  " in final:
        final = final.replace("  ", " ")
    return (final.lower())

def clean_docs(docs, stops):
    months = load_data("data/months.json")
    stops = stops+months
    final = []
    for doc in docs:
        clean_doc = remove_stops(doc, stops)
        final.append(clean_doc)
    return (final)

In [4]:
data = load_data("data/corpus.json")["data"]
print (data[0])

Age is an issue of mind over matter. If you don't mind, it doesn't matter.


In [5]:
cleaned_docs = clean_docs(data, stopwords)
print (cleaned_docs[0])

age issue mind matter mind matter


In [6]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)


lemmatized_texts = lemmatization(cleaned_docs)
print (lemmatized_texts[0])

age issue mind matter mind matter


In [7]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)

print (data_words[0])

['age', 'issue', 'mind', 'matter', 'mind', 'matter']


In [8]:
#BIGRAMS AND TRIGRAMS
bigram_phrases = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram_phrases = gensim.models.Phrases(bigram_phrases[data_words], threshold=100)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return([bigram[doc] for doc in texts])

def make_trigrams(texts):
    return ([trigram[bigram[doc]] for doc in texts])

data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigrams(data_bigrams)

print (data_bigrams_trigrams[0])

['age', 'issue', 'mind', 'matter', 'mind', 'matter']


In [9]:
#TF-IDF REMOVAL
from gensim.models import TfidfModel

id2word = corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

corpus = [id2word.doc2bow(text) for text in texts]
print (corpus[0])

tfidf = TfidfModel(corpus, id2word=id2word)

low_value = 0.03
words  = []
words_missing_in_tfidf = []
for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = [] #reinitialize to be safe. You can skip this.
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words+words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow

[(0, 1), (1, 1), (2, 2), (3, 2)]


In [10]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=30,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")


In [ ]:
# Getting documents by topic
all_topics = lda_model.print_topics()
docs_per_topic = [[] for _ in all_topics]

# now, for every doc...
for doc_id, doc_bow in enumerate(corpus):
    # ...get its lda_model...
    doc_topics = lda_model.get_document_topics(doc_bow)
    # ...& for each of its topics...
    for topic_id, score in doc_topics:
        if topic_id < 20:
            docs_per_topic[topic_id].append((doc_id, score))

for doc_list in docs_per_topic:
    doc_list.sort(key=lambda id_and_score: id_and_score[1], reverse=True)

for topic in all_topics:
    # print(f"Document: {doc_id}, Score: {score}")
    print("----------------")
    print(topic)
    print("----------------")
    print()

# print(docs_per_topic[0][:10])
print()
# print(data[5293])

----------------
(24, '0.000*"legitimize" + 0.000*"frill" + 0.000*"kickboxe" + 0.000*"residency" + 0.000*"internship" + 0.000*"blander" + 0.000*"radishe" + 0.000*"matchstick" + 0.000*"hor" + 0.000*"northwest"')
----------------

----------------
(9, '0.000*"legitimize" + 0.000*"frill" + 0.000*"kickboxe" + 0.000*"residency" + 0.000*"internship" + 0.000*"blander" + 0.000*"radishe" + 0.000*"matchstick" + 0.000*"hor" + 0.000*"northwest"')
----------------

----------------
(13, '0.310*"study" + 0.097*"determine" + 0.050*"valuable" + 0.000*"wisdom" + 0.000*"residency" + 0.000*"kickboxe" + 0.000*"internship" + 0.000*"radishe" + 0.000*"jamaican" + 0.000*"shkoff"')
----------------

----------------
(10, '0.427*"age" + 0.196*"consider" + 0.077*"youth" + 0.000*"wisdom" + 0.000*"radishe" + 0.000*"matchstick" + 0.000*"hor" + 0.000*"residency" + 0.000*"blander" + 0.000*"kickboxe"')
----------------

----------------
(25, '0.441*"real" + 0.108*"own" + 0.078*"grace" + 0.047*"add" + 0.021*"vice" + 0.

## Vizualizing the Data

In [14]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

/home/simeon/.local/lib/python3.10/site-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
2     -0.485188  0.159369       1        1  7.945114
8      0.169480 -0.443986       2        1  6.396519
28     0.421433  0.203793       3        1  6.065728
5      0.338467  0.371072       4        1  5.908543
23     0.453221 -0.160327       5        1  5.500739
4     -0.003116 -0.452921       6        1  5.337230
17    -0.282215  0.376958       7        1  4.406544
19     0.449713  0.048733       8        1  4.263131
16    -0.382088 -0.296568       9        1  4.251389
22    -0.482274 -0.045876      10        1  3.956056
18    -0.305906  0.251857      11        1  3.939186
0     -0.171963 -0.505681      12        1  3.853217
7     -0.197697 -0.308527      13        1  3.730546
6      0.343238 -0.039434      14        1  3.721442
21     0.024680  0.470645      15        1  3.623945
26     0.376228 -0.361910      16        1  3.576759
20     0.243058 -0.229744      17        1  3.574653
29    -0.301359  0.075590      18        1  3.090852
27    -0.374532 -0.156353      19        1  2.810880
1      0.080713  0.362819      20        1  2.455456
14     0.193987  0.289796      21        1  2.124381
12    -0.071777  0.216024      22        1  2.025215
11    -0.121551  0.387558      23        1  1.680271
3      0.064592 -0.230914      24        1  1.583476
15     0.188143  0.113367      25        1  1.518695
25    -0.146731 -0.123386      26        1  1.136170
10     0.028476 -0.099167      27        1  1.095400
13    -0.061779  0.043335      28        1  0.334189
24     0.006373  0.041938      29        1  0.047137
9      0.006373  0.041939      30        1  0.047137, topic_info=         Term          Freq         Total Category  logprob  loglift
152    people  31443.000000  31443.000000  Default  30.0000  30.0000
162      work  24256.000000  24256.000000  Default  29.0000  29.0000
7        life  24021.000000  24021.000000  Default  28.0000  28.0000
28      woman  19938.000000  19938.000000  Default  27.0000  27.0000
151      love  19332.000000  19332.000000  Default  26.0000  26.0000
..        ...           ...           ...      ...      ...      ...
23       true      0.014306   4242.934886  Topic30 -10.0033  -4.9402
25   birthday      0.014306    449.087764  Topic30 -10.0033  -2.6944
27   remember      0.014306   2145.416812  Topic30 -10.0033  -4.2583
28      woman      0.014306  19938.919034  Topic30 -10.0033  -6.4876
29       grow      0.014306   3942.732623  Topic30 -10.0033  -4.8668

[1413 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
130       4  0.973118  absurd
47       26  0.990629     add
318       7  0.995836  admire
182      11  0.996364   adult
131       4  0.995881  advice
...     ...       ...     ...
162      12  0.999871    work
105       6  0.999233   world
58       19  0.999596    year
12        9  0.999141   young
66       27  0.994229   youth

[234 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 9, 29, 6, 24, 5, 18, 20, 17, 23, 19, 1, 8, 7, 22, 27, 21, 30, 28, 2, 15, 13, 12, 4, 16, 26, 11, 14, 25, 10])